In [ ]:
from typing import Tuple

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import pandas as pd
import torch.nn.functional as F

In [ ]:
def fit_model(model: nn.Module, input_data: torch.Tensor, labels: torch.Tensor, epoches: int = 10,
              lr=0.001, batch_size: int = 32):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    dataset = TensorDataset(input_data, labels)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    for epoch in range(epoches):
        for data, target in loader:
            optimizer.zero_grad()
            output = model(data.float()).squeeze()
            loss = criterion(output, target.float())
            loss.backward()
            optimizer.step()
        print(f'Epoch {epoch + 1}, Loss: {loss.item()}')

    return model


def load_model(model: nn.Module, filepath: str):
    model.load_state_dict(torch.load(filepath))
    return model


def dataframe_to_tensor(df: pd.DataFrame) -> Tuple[torch.Tensor, torch.Tensor]:
    features = torch.tensor(df.values, dtype=torch.float32).to('cuda')
    return features

def list_to_tensor(lst: List) -> torch.Tensor:
    return  torch.tensor(lst, dtype=torch.float32).to('cuda')

def tensor_to_list(tensor: torch.Tensor) -> list:
    return tensor.detach().cpu().tolist()

In [ ]:
class AttentionModel(nn.Module):
    def __init__(self, input_size: int, hidden_sizes: Optional[List[int]] = None,
                 output_size: int = 1):
        super(AttentionModel, self).__init__()
        hidden_sizes = hidden_sizes or [30]
        # Веса внимания для входных фич
        self.attention_weights = nn.Parameter(torch.randn(input_size, requires_grad=True))

        self.layer1 = nn.Linear(input_size, hidden_sizes[0])
        self.layer2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.layer3 = nn.Linear(hidden_sizes[1], output_size)

    def forward(self, x):
        x = F.relu(self.layer1(x))
        x = F.relu(self.layer2(x))
        x = self.layer3(x)
        return x

In [ ]:
input_size = 4
columns = [f'x{i + 1}' for i in range(input_size)]
train_data = pd.DataFrame(np.random.randint(1, 100, size=(10000, input_size)), columns=columns)

target_data = train_data['x1'] + train_data['x2'] - train_data['x3']

input_data = dataframe_to_tensor(train_data)
labels = dataframe_to_tensor(target_data)

filename = 'att'

In [ ]:
# Загрузка модели из файла
model = load_model(AttentionModel(input_size), f'{filename}.pth').to('cuda')

In [ ]:
# Обучение модели
model = AttentionModel(input_size, [128, 128], 1)

model = model.to('cuda')
fit_model(model, input_data, labels, epoches=50, batch_size=32)
torch.save(model.state_dict(), f'{filename}.pth')  # Сохранение весов модели

In [ ]:
test_data = [4, 5, 6, 7]

model(list_to_tensor(test_data))